In [ ]:
import numpy as np

In [ ]:
def gauss_method(n, directions):
    """
    Implements Gauss's method for orbit determination given n measurements of direction vectors
    
    Args:
    n (int): The number of measurements
    directions (list): A list of n direction vectors, where each vector is represented as a 3-tuple of floats
    
    Returns:
    tuple: A tuple containing the estimated position and velocity vectors of the object
    """
    A = np.zeros((n, 3))
    for i in range(n):
        A[i] = directions[i]
    
    ATA = np.matmul(A.T, A)
    U, s, V = np.linalg.svd(ATA)
    Q = np.matmul(U, V.T)
    X = np.matmul(Q, directions)
    r = np.linalg.norm(X)
    v = np.cross(X, np.cross(directions[0], X)) / r
    return X, v

In [ ]:
def gauss_method_time(n, directions, times):
    """
    Implements Gauss's method for orbit determination given n measurements of direction vectors and the time of each measurement
    
    Args:
    n (int): The number of measurements
    directions (list): A list of n direction vectors, where each vector is represented as a 3-tuple of floats
    times (list): A list of n floats representing the time of each measurement
    
    Returns:
    tuple: A tuple containing the estimated position and velocity vectors of the object
    """
    A = np.zeros((n, 3))
    for i in range(n):
        A[i] = directions[i]
    
    ATA = np.matmul(A.T, A)
    U, s, V = np.linalg.svd(ATA)
    Q = np.matmul(U, V.T)
    X = np.matmul(Q, directions)
    r = np.linalg.norm(X)
    
    B = np.zeros((n-1, 4))
    for i in range(n-1):
        delta_t = times[i+1] - times[0]
        r_i = np.linalg.norm(X - A[i+1]*r)
        B[i][0] = delta_t
        B[i][1] = A[i+1][0]*r_i
        B[i][2] = A[i+1][1]*r_i
        B[i][3] = A[i+1][2]*r_i
        
    BTB = np.matmul(B.T, B)
    w, v = np.linalg.eig(BTB)
    min_eigval = np.min(w)
    if min_eigval < 0:
        print("Error: BTB matrix is not positive definite")
        return None
    u = v[:, np.argmin(w)]
    v_i = u[1:4]
    v_f = u[0]
    v = np.concatenate(([v_f], v_i))
    v = v / np.linalg.norm(v)
    v = v * np.sqrt(min_eigval)
    
    return X, v